# 外键

## 介绍
外键是一个表中的一列或一组列，它唯一标识另一个表中的行。外键通过引用主键或所引用表的唯一约束在两个表中的数据之间建立链接。包含外键的表称为引用表或子表。相反，被外键引用的表称为被引用表或父表。

## 外键的作用
外键的主要作用是维护关系数据库的参照完整性，确保父表和子表之间的关系有效。此外，外键还能在父表发生变化时自动更新或删除子表中的相关行，从而保持一致性。

```
[CONSTRAINT fk_name]
   FOREIGN KEY(fk_columns) 
   REFERENCES parent_table(parent_key_columns)
   [ON DELETE delete_action]
   [ON UPDATE update_action]
```


In [1]:
import common.ipynb_importer
from db.pg.pg_00_common import *

cursor = pg_connect()

importing Jupyter notebook from D:\sourcecode\keep_learning\db\pg\pg_00_common.ipynb


# NO ACTION

In [2]:
sql = """
DROP TABLE IF EXISTS customers;
DROP TABLE IF EXISTS contacts;

CREATE TABLE customers(
   customer_id INT GENERATED ALWAYS AS IDENTITY,
   customer_name VARCHAR(255) NOT NULL,
   PRIMARY KEY(customer_id)
);

CREATE TABLE contacts(
   contact_id INT GENERATED ALWAYS AS IDENTITY,
   customer_id INT,
   contact_name VARCHAR(255) NOT NULL,
   phone VARCHAR(15),
   email VARCHAR(100),
   PRIMARY KEY(contact_id),
   CONSTRAINT fk_customer
      FOREIGN KEY(customer_id) 
        REFERENCES customers(customer_id)
);
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x27ecd1b8930>

In [3]:
sql = """
INSERT INTO customers(customer_name)
VALUES('BlueBird Inc'),
      ('Dolphin LLC');
	   
INSERT INTO contacts(customer_id, contact_name, phone, email)
VALUES(1,'John Doe','(408)-111-1234','john.doe@bluebird.dev'),
      (1,'Jane Doe','(408)-111-1235','jane.doe@bluebird.dev'),
      (2,'David Wright','(408)-222-1234','david.wright@dolphin.dev');
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x27ecd1b8930>

In [4]:
sql = """
SELECT * FROM contacts;
"""
run_sql(cursor, sql)

   contact_id  customer_id  contact_name           phone  \
0           1            1      John Doe  (408)-111-1234   
1           2            1      Jane Doe  (408)-111-1235   
2           3            2  David Wright  (408)-222-1234   

                      email  
0     john.doe@bluebird.dev  
1     jane.doe@bluebird.dev  
2  david.wright@dolphin.dev  


In [5]:
# 由于执行了 ON DELETE NO ACTION 操作，PostgreSQL 出现了违反约束的情况，因此删除操作失败。
sql = """
DELETE FROM customers
WHERE customer_id = 1;
"""
# run_sql(cursor, sql)

# SET NULL

In [6]:
sql = """
DROP TABLE IF EXISTS contacts;
DROP TABLE IF EXISTS customers;

CREATE TABLE customers(
   customer_id INT GENERATED ALWAYS AS IDENTITY,
   customer_name VARCHAR(255) NOT NULL,
   PRIMARY KEY(customer_id)
);

CREATE TABLE contacts(
   contact_id INT GENERATED ALWAYS AS IDENTITY,
   customer_id INT,
   contact_name VARCHAR(255) NOT NULL,
   phone VARCHAR(15),
   email VARCHAR(100),
   PRIMARY KEY(contact_id),
   CONSTRAINT fk_customer
      FOREIGN KEY(customer_id) 
	  REFERENCES customers(customer_id)
	  ON DELETE SET NULL
);

INSERT INTO customers(customer_name)
VALUES('BlueBird Inc'),
      ('Dolphin LLC');	   
	   
INSERT INTO contacts(customer_id, contact_name, phone, email)
VALUES(1,'John Doe','(408)-111-1234','john.doe@bluebird.dev'),
      (1,'Jane Doe','(408)-111-1235','jane.doe@bluebird.dev'),
      (2,'David Wright','(408)-222-1234','david.wright@dolphin.dev');
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x27ecd1b8930>

In [9]:
# 由于执行了 ON DELETE SET NULL 操作，PostgreSQL 将 contacts 表中的 customer_id 设置为 NULL。
sql = """
DELETE FROM customers
WHERE customer_id = 1;
"""
cursor.execute(sql)

sql = """
SELECT * FROM contacts;
"""
run_sql(cursor, sql)

   contact_id  customer_id  contact_name           phone  \
0           3          2.0  David Wright  (408)-222-1234   
1           1          NaN      John Doe  (408)-111-1234   
2           2          NaN      Jane Doe  (408)-111-1235   

                      email  
0  david.wright@dolphin.dev  
1     john.doe@bluebird.dev  
2     jane.doe@bluebird.dev  


# CASCADE

In [10]:
sql = """
DROP TABLE IF EXISTS contacts;
DROP TABLE IF EXISTS customers;

CREATE TABLE customers(
   customer_id INT GENERATED ALWAYS AS IDENTITY,
   customer_name VARCHAR(255) NOT NULL,
   PRIMARY KEY(customer_id)
);

CREATE TABLE contacts(
   contact_id INT GENERATED ALWAYS AS IDENTITY,
   customer_id INT,
   contact_name VARCHAR(255) NOT NULL,
   phone VARCHAR(15),
   email VARCHAR(100),
   PRIMARY KEY(contact_id),
   CONSTRAINT fk_customer
      FOREIGN KEY(customer_id) 
	  REFERENCES customers(customer_id)
	  ON DELETE CASCADE
);

INSERT INTO customers(customer_name)
VALUES('BlueBird Inc'),
      ('Dolphin LLC');	   
	   
INSERT INTO contacts(customer_id, contact_name, phone, email)
VALUES(1,'John Doe','(408)-111-1234','john.doe@bluebird.dev'),
      (1,'Jane Doe','(408)-111-1235','jane.doe@bluebird.dev'),
      (2,'David Wright','(408)-222-1234','david.wright@dolphin.dev');
"""
cursor.execute(sql)

In [12]:
# 由于执行了 ON DELETE CASCADE 操作，PostgreSQL 删除了 customers 表中 customer_id 为 1 的行，并且删除了 contacts 表中 customer_id 为 1 的行。   
sql = """
DELETE FROM customers
WHERE customer_id = 1;
"""
cursor.execute(sql)

sql = """
SELECT * FROM contacts;
"""
run_sql(cursor, sql)

   contact_id  customer_id  contact_name           phone  \
0           3            2  David Wright  (408)-222-1234   

                      email  
0  david.wright@dolphin.dev  


# SET DEFAULT
要在现有表中添加外键约束，需要使用下面形式的 ALTER TABLE 语句：
```
ALTER TABLE child_table 
ADD CONSTRAINT constraint_name 
FOREIGN KEY (fk_columns) 
REFERENCES parent_table (parent_key_columns);
```

将带有 `ON DELETE CASCADE` 选项的外键约束添加到现有表时：
```
ALTER TABLE child_table
DROP CONSTRAINT constraint_fkey;
```
然后：
```
ALTER TABLE child_table
ADD CONSTRAINT constraint_fk
FOREIGN KEY (fk_columns)
REFERENCES parent_table(parent_key_columns)
ON DELETE CASCADE;
```